In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from numpy import expand_dims
from keras.models import load_model
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from matplotlib import pyplot
from matplotlib.patches import Rectangle

In [2]:
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

In [3]:
# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

In [4]:
# make suare images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

print("x_train shape: ", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

x_train shape:  (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [5]:
# Convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [6]:
# load dense softmax model
model = load_model('./Data/conv_flatten_softmax_model.h5')

2022-01-30 16:25:22.598448: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
a_val_img = x_test[0]

In [8]:
# Cool
model.predict(np.array([a_val_img]))

array([[1.1773768e-21, 9.4355975e-19, 1.9853956e-14, 3.2595419e-16,
        6.6482707e-28, 1.7441588e-19, 6.9608934e-37, 1.0000000e+00,
        1.4679344e-18, 6.7851879e-14]], dtype=float32)

In [10]:
import tf2onnx
import onnxruntime as rt

In [11]:
spec = (tf.TensorSpec((None, 28, 28, 1), tf.float32, name="input"),)
output_path = './Data/conv_flatten_softmax_model.onnx'

model_proto, _ = tf2onnx.convert.from_keras(model, input_signature=spec, opset=13, output_path=output_path)
output_names = [n.name for n in model_proto.graph.output]

Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`


2022-01-30 16:26:13.840457: I tensorflow/core/grappler/devices.cc:75] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2022-01-30 16:26:13.841990: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1149] Optimization results for grappler item: graph_to_optimize
  function_optimizer: function_optimizer did nothing. time = 0.006ms.
  function_optimizer: function_optimizer did nothing. time = 0.001ms.

2022-01-30 16:26:13.879979: I tensorflow/core/grappler/devices.cc:75] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2022-01-30 16:26:13.889796: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1149] Optimization results for grappler item: graph_to_optimize
  constant_folding: Graph size after: 25 nodes (-8), 32 edges (-8), time = 4.604ms.
  function_optimizer: function_optimizer did nothing. time = 0.263ms.
 

In [12]:
# Run the ONNX model
providers = ['CPUExecutionProvider']
m = rt.InferenceSession(output_path, providers=providers)
onnx_pred = m.run(output_names, {"input": [a_val_img]})

In [13]:
onnx_pred[0]

array([[1.1773812e-21, 9.4357061e-19, 1.9854069e-14, 3.2595670e-16,
        6.6482201e-28, 1.7441788e-19, 6.9609997e-37, 1.0000000e+00,
        1.4679399e-18, 6.7852401e-14]], dtype=float32)

In [14]:
model.predict(np.array([a_val_img]))

array([[1.1773768e-21, 9.4355975e-19, 1.9853956e-14, 3.2595419e-16,
        6.6482707e-28, 1.7441588e-19, 6.9608934e-37, 1.0000000e+00,
        1.4679344e-18, 6.7851879e-14]], dtype=float32)